In [3]:
import csv
import json
import unicodedata
import numpy as np
import networkx as nx

import requests
import pandas as pd
from pattern import web
import matplotlib.pyplot as plt
import re

from matplotlib import rcParams
from bs4 import BeautifulSoup
from copy import deepcopy

In [4]:
pd.set_option('display.max_columns', None)


In [5]:
def n_mean(df,colname,k_num):
    response_vals=list(df[colname])
    n_means_res=n_mean_help(response_vals,k_num)
    lasts=[None]*(k_num)
    n_means_res.extend(lasts)
    n_means_res.pop(0)
    return n_means_res

In [6]:
def n_mean_help(alist,k_num):
    n_means=[]
    sum_k=sum(alist[:k_num])
    n_means.append(sum_k)
    for ith in range(k_num,len(alist)):
        sum_k=alist[ith]+sum_k
        sum_k=-alist[ith-k_num]+sum_k
        n_means.append(sum_k)
    return n_means

In [104]:
df_frames=[]
for year in range(1989,2016):
    games_data=pd.read_csv('player_games_data_'+str(year)+'.csv',index_col=False)

    del games_data['Unnamed: 0']

    games_data=games_data.sort(['draft_years', 'Drafts','Year','ith_game'])
    games_data=games_data.reset_index(drop=True)

    games_data['gm_result']=games_data['gm_result'].replace(['W'], 1)
    games_data['gm_result']=games_data['gm_result'].replace(['L'], 0)

    games_data['gm_loc']=games_data['gm_loc'].replace(['Home'], 1)
    games_data['gm_loc']=games_data['gm_loc'].replace(['Away'], 0)

    games_data['fc_win'].fillna(0, inplace=True)
    games_data['fc_loss'].fillna(0, inplace=True)
    
    df_frames.append(games_data)
    

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [105]:
pd.set_option('display.max_columns', None)
new_player_data=pd.concat(df_frames)

In [106]:
# file_name_player='Data/Game_Data/clean_data/all_games_data.csv'
# new_player_data=pd.read_csv(file_name_player,index_col=False)
# del new_player_data['Unnamed: 0']

In [107]:
# list(new_player_data.columns)

In [108]:
new_player_data=new_player_data.rename(columns = {'ith_game':'num_games'})
new_player_data=new_player_data.rename(columns = {'team_for':'tm_for'})
new_player_data=new_player_data.rename(columns = {'team_agt':'tm_agt'})
new_player_data=new_player_data.rename(columns = {'date_game':'dates_game'})
new_player_data=new_player_data.rename(columns = {'gm_loc':'player_gm_loc'})
new_player_data=new_player_data.rename(columns = {'num_games':'player_num_games'})
new_player_data=new_player_data.rename(columns = {'gm_result':'player_gm_result'})

In [109]:
new_player_data.shape

(742536, 32)

In [110]:
new_player_data.head(68)

,player_gm_result,age,player_num_games,tm_for,tm_agt,dates_game,player_gm_loc,goals_all,assists_all,plus_minus,pen_min,goals_ev,goals_pp,goals_sh,assists_pp,assists_ev,assists_sh,num_shifts,time_on_ice,hits_all,blocks_all,fc_win,fc_loss,Year,Name,Url,Height,Weight,ShotHand,Position,Drafts,draft_years
0,0.0,19.64,1.0,QUE,HAR,1990-10-04,0.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
1,0.0,19.64,2.0,QUE,BOS,1990-10-06,0.0,0.0,0.0,-2.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
2,1.0,19.65,3.0,QUE,BOS,1990-10-07,1.0,0.0,0.0,-1.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
3,1.0,19.65,4.0,QUE,TOR,1990-10-10,0.0,1.0,3.0,2.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
4,0.0,19.66,5.0,QUE,BUF,1990-10-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
5,1.0,19.66,6.0,QUE,BUF,1990-10-13,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
6,0.0,19.67,7.0,QUE,HAR,1990-10-16,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
7,0.0,19.68,8.0,QUE,PHI,1990-10-18,0.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
8,0.0,19.68,9.0,QUE,DET,1990-10-20,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0
9,0.0,19.68,10.0,QUE,VAN,1990-10-21,1.0,1.0,0.0,-1.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1991.0,Mats Sundin,s/sundima01,196,231,R,C,1.0,1989.0


In [94]:
df_frame_teams=[]

for year in range(1989,2016):
    new_team_data_home=pd.read_csv('team_data_games_data_'+str(year)+'.csv',index_col=False)
    del new_team_data_home['Unnamed: 0']

    new_team_data_home=new_team_data_home.rename(columns = {'gm_loc':'tm_gm_loc'})
    new_team_data_home=new_team_data_home.rename(columns = {'num_games':'tm_num_games'})
    new_team_data_home=new_team_data_home.rename(columns = {'gm_result':'tm_gm_result'})

    new_team_data_away=deepcopy(new_team_data_home)
    new_team_data_home['tm']=new_team_data_home['tm_agt']
    new_team_data_away['tm']=new_team_data_away['tm_for']

    merge_team_data=pd.merge(new_team_data_home, new_team_data_away, on=['tm','dates_game'], how='inner',suffixes=('_home', '_away'))

    del merge_team_data['tm_for_away']
    del merge_team_data['tm_agt_away']
    del merge_team_data['tm']
    merge_team_data=merge_team_data.rename(columns = {'tm_for_home':'tm_for'})
    merge_team_data=merge_team_data.rename(columns = {'tm_agt_home':'tm_agt'})
    df_frame_teams.append(merge_team_data)
    
    

In [95]:
merge_team_data.head()

,tm_for,tm_num_games_home,dates_game,tm_agt,tm_gm_loc_home,tm_goals_for_home,tm_goals_agt_home,tm_gm_result_home,cum_wins_for_home,cum_loss_for_home,tm_loss_home,tm_shots_for_home,tm_pen_min_home,tm_goals_pp_for_home,tm_goals_sh_for_home,tm_shots_agt_home,opp_pen_min_home,tm_goals_agt_pp_home,tm_goals_agt_sh_home,tm_num_games_away,tm_gm_loc_away,tm_goals_for_away,tm_goals_agt_away,tm_gm_result_away,cum_wins_for_away,cum_loss_for_away,tm_loss_away,tm_shots_for_away,tm_pen_min_away,tm_goals_pp_for_away,tm_goals_sh_for_away,tm_shots_agt_away,opp_pen_min_away,tm_goals_agt_pp_away,tm_goals_agt_sh_away
0,ANA,1,2014-10-09,PIT,A,4,6,L,0,1,L 1,29,13,3,0,39,15,1,1,1,H,6,4,W,1,0,W 1,39,15,1,1,29,13,3,0
1,ANA,2,2014-10-11,DET,A,3,2,W,1,1,W 1,26,6,0,0,29,8,0,0,2,H,2,3,L,1,1,L 1,29,8,0,0,26,6,0,0
2,ANA,3,2014-10-13,BUF,A,5,1,W,2,1,W 2,44,13,1,0,12,19,0,0,3,H,1,5,L,0,3,L 3,12,19,0,0,44,13,1,0
3,ANA,4,2014-10-14,PHI,A,4,3,W,3,1,W 3,28,8,0,0,42,4,2,0,4,H,3,4,L,0,2,L 4,42,4,2,0,28,8,0,0
4,ANA,5,2014-10-17,MIN,H,2,1,W,4,1,W 4,26,13,0,1,28,25,0,0,3,A,1,2,L,2,1,L 1,28,25,0,0,26,13,0,1


In [96]:
merge_team_data_all=pd.concat(df_frame_teams)

In [97]:
merge_team_data_all.head()

,cum_loss_for_away,cum_loss_for_home,cum_wins_for_away,cum_wins_for_home,dates_game,opp_pen_min_away,opp_pen_min_home,tm_agt,tm_for,tm_gm_loc_away,tm_gm_loc_home,tm_gm_result_away,tm_gm_result_home,tm_goals_agt_away,tm_goals_agt_home,tm_goals_agt_pp_away,tm_goals_agt_pp_home,tm_goals_agt_sh_away,tm_goals_agt_sh_home,tm_goals_for_away,tm_goals_for_home,tm_goals_pp_for_away,tm_goals_pp_for_home,tm_goals_sh_for_away,tm_goals_sh_for_home,tm_loss_away,tm_loss_home,tm_num_games_away,tm_num_games_home,tm_pen_min_away,tm_pen_min_home,tm_shots_agt_away,tm_shots_agt_home,tm_shots_for_away,tm_shots_for_home
0,1.0,0.0,0.0,1.0,1988-10-06,4.0,6.0,TOR,BOS,A,H,L,W,2.0,1.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,L 1,W 1,1.0,1.0,6.0,4.0,27.0,18.0,18.0,27.0
1,2.0,0.0,0.0,2.0,1988-10-08,26.0,28.0,HAR,BOS,H,A,L,W,6.0,2.0,3.0,2.0,0.0,0.0,2.0,6.0,2.0,3.0,0.0,0.0,L 2,W 2,2.0,2.0,28.0,26.0,21.0,22.0,22.0,21.0
2,3.0,0.0,0.0,3.0,1988-10-09,19.0,23.0,HAR,BOS,A,H,L,W,3.0,1.0,1.0,1.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,0.0,L 3,W 3,3.0,3.0,23.0,19.0,40.0,30.0,30.0,40.0
3,0.0,1.0,4.0,3.0,1988-10-12,8.0,8.0,LAK,BOS,H,A,W,L,2.0,6.0,0.0,0.0,0.0,1.0,6.0,2.0,0.0,0.0,1.0,0.0,W 4,L 1,4.0,4.0,8.0,8.0,35.0,36.0,36.0,35.0
4,4.0,2.0,1.0,3.0,1988-10-15,12.0,10.0,MNS,BOS,H,A,W,L,1.0,5.0,1.0,0.0,0.0,0.0,5.0,1.0,0.0,1.0,0.0,0.0,W 1,L 2,5.0,5.0,10.0,12.0,36.0,25.0,25.0,36.0


In [98]:
merge_data=pd.merge(new_player_data, merge_team_data_all, on=['tm_for','tm_agt','dates_game'], how='inner')

In [99]:
merge_data=merge_data.sort(['draft_years','Drafts' , 'dates_game'], ascending=[1, 1, 1])

/Users/takehiromatsuzawa/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [100]:
merge_data=merge_data.reset_index(drop=True)

In [101]:
merge_data.shape

(710790, 64)

In [102]:
merge_data.to_csv('draft_merged_data.csv')